In [103]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [104]:
import pandas as pd

## 독거노인과 행정동별 병원

### Medical Data 전처리

In [152]:
csv_path_medical = '/content/drive/MyDrive/town_cc_big_data/data/강원도 춘천시_의료기관약국_20220805.csv'
df_medical_raw = pd.read_csv(csv_path_medical, encoding='euc-kr')

In [153]:
address_data = df_medical_raw['도로명주소']
kind_data = df_medical_raw['종류']

location = []
kind = []

for i in range(len(address_data)):
    split_data = address_data[i].split()
    if len(split_data) >= 3:
        if split_data[2][-1] == '면':
          location.append(split_data[2])
          kind.append(kind_data[i])
    if len(split_data) >= 5 and split_data[2][-1] != '면' and len(split_data[-1]) >= 2:
      if split_data[-1][-2] == '동':
        location.append(split_data[-1][1:-1])
        kind.append(kind_data[i])

In [154]:
df_medical = pd.DataFrame({'종류': kind, '행정동': location})
df_medical

,종류,행정동
0,병원,효자동
1,병원,남면
2,병원,석사동
3,병원,효자동
4,병원,동면
...,...,...
377,한의원,온의동
378,한의원,퇴계동
379,한의원,약사동
380,한의원,효자동


### 행정동별 병원 수 카운트

In [155]:
count_by_dong = df_medical.groupby('행정동').count().rename(columns={'종류': '병원 수'})
count_by_dong = count_by_dong.sort_values('병원 수', ascending=True)
count_by_dong

,병원 수
행정동,
신동,1
남면,1
요선동,1
봉의동,1
삼천동,1
약사동,1
칠전동,2
남산면,2
동산면,2


---

더 자세한 분석을 위해서는 면적이랑 같이 해당 데이터를 봐야 할 듯.    
그리고 아예 없는 곳도 있는지 확인해봐야 함.

### 독거노인_데이터_전처리

In [156]:
csv_path_old = '/content/drive/MyDrive/town_cc_big_data/data/강원도 춘천시_독거노인현황_20220620.csv'
df_old_raw = pd.read_csv(csv_path_old, encoding='euc-kr')

In [157]:
df_old_raw

,시도명,시군구명,행정동명,연령,인구수,남자인구수,여자인구수,데이터기준일자
0,강원도,춘천시,신북읍,65 - 69,156,78,78,2022-06-20
1,강원도,춘천시,신북읍,70 - 74,109,49,60,2022-06-20
2,강원도,춘천시,신북읍,75 - 79,114,28,86,2022-06-20
3,강원도,춘천시,신북읍,80 - 84,139,36,103,2022-06-20
4,강원도,춘천시,신북읍,85 - 89,86,15,71,2022-06-20
...,...,...,...,...,...,...,...,...
180,강원도,춘천시,신사우동,80 - 84,202,50,152,2022-06-20
181,강원도,춘천시,신사우동,85 - 89,121,29,92,2022-06-20
182,강원도,춘천시,신사우동,90 - 94,30,1,29,2022-06-20
183,강원도,춘천시,신사우동,95 - 99,14,4,10,2022-06-20


In [158]:
df_old = df_old_raw.groupby('행정동명')['인구수'].sum().reset_index()
df_old = df_old.sort_values('인구수', ascending = False)
df_old

,행정동명,인구수
18,퇴계동,1538
11,석사동,1316
24,후평3동,1168
15,신사우동,1044
20,효자2동,1019
22,후평1동,848
0,강남동,799
6,동면,769
5,동내면,677
14,신북읍,643


이걸로는 부족하고 전체 인구수를 찾아봐서, 전체인구수 대비 독거노인 비율을 찾아야 할 듯.   
비율이 높은데 의료기관이 적은거면 문제가 될 수 있기 때문.       
~~그리고 도대체 여기에 없는 동들은 어디서 데이터를 추출해온거야?~~

일단 남산면이랑 동산면이 취약하긴 함.      
독거노인이 450명 넘게 있는데 의료기관이 2개 밖에 없음 실화냐    
-> 데이터 추출을 제대로 했는지 한 번 봐야 될 듯

----

## 1인 가구와 정신건강 관련 서비스 제공처

### 정신건강 관련 서비스 제공처

In [112]:
csv_path_mental1 = '/content/drive/MyDrive/town_cc_big_data/data/강원도 춘천시_정신건강센터 현황_20220822.csv'
csv_path_mental2 = '/content/drive/MyDrive/town_cc_big_data/data/강원도 춘천시_정신건강의학과 현황_20220822.csv'

In [113]:
df_old_mental1 = pd.read_csv(csv_path_mental1, encoding='euc-kr')
df_old_mental2 = pd.read_csv(csv_path_mental2, encoding='euc-kr')

In [114]:
df_old_mental1

,유형,기관명,도로명주소,전화번호,운영유형,위도,경도,데이터기준일
0,공공기관,춘천시정신건강복지센터,강원도 춘천시 효제길 35,033-244-7574,기초정신건강복지센터,37.871309,127.737216,2022-08-22
1,공공기관,춘천시중독관리통합지원센터,"강원도 춘천시 삭주로 84, 3층",033-240-5732,중독관리통합지원센터,37.883399,127.740265,2022-08-22
2,공공기관,우리내꿈터,"강원도 춘천시 중앙로 104-1, 2층",033-255-0661,정신재활시설,37.877215,127.723224,2022-08-22


In [115]:
df_old_mental2

,상호명,도로명주소,전화번호,위도,경도,데이터기준일
0,강원대학교병원,강원도 춘천시 백령로156,033-258-2000,37.874705,127.744797,2022-08-22
1,국립춘천병원,강원도 춘천시 동산면 영서로824,033-260-3000,37.771500,127.766717,2022-08-22
2,춘천서인병원,강원도 춘천시 동산면 새술막길 315-77,033-260-7920,37.770571,127.728013,2022-08-22
3,춘천예현병원,강원도 춘천시 서면 샛말길 122-11,033-264-9885,37.851304,127.617334,2022-08-22
4,봄내병원,강원도 춘천시 춘천로180,033-255-8515,37.876222,127.734095,2022-08-22
5,베드로병원,강원도 춘천시 공지로270,033-255-0085,37.865971,127.735371,2022-08-22
6,한림대학교춘천성심병원,강원도 춘천시 삭주로77,033-240-5000,37.884130,127.740808,2022-08-22
7,춘천정신건강의학과의원,강원도 춘천시 퇴계로184,033-264-0507,37.857348,127.739129,2022-08-22
8,춘천마음벗의원,강원도 춘천시 후석로326,033-255-0300,37.880069,127.750190,2022-08-22
9,안정신건강의학과의원,강원도 춘천시 금강로68-2,033-244-8333,37.879237,127.728651,2022-08-22


어? 근데 진짜 너무 없는데;;; → 저 14개 데이터 직접 행정동이랑 구로 매핑 시키기

### 1인 가구 수

In [116]:
csv_path_one_person = '/content/drive/MyDrive/town_cc_big_data/data/강원도 춘천시_읍면동별 1인 가구수_20220816.csv'

In [117]:
df_one_person = pd.read_csv(csv_path_one_person, encoding='euc-kr')
df_one_person

,행정동,기준년도,1인가구수,데이터기준일
0,신북읍,2010,1051,2022-08-16
1,동면,2010,1194,2022-08-16
2,동산면,2010,292,2022-08-16
3,신동면,2010,319,2022-08-16
4,남면,2010,284,2022-08-16
...,...,...,...,...
95,효자3동,2021,1707,2022-08-16
96,석사동,2021,5610,2022-08-16
97,퇴계동,2021,6192,2022-08-16
98,강남동,2021,3401,2022-08-16


In [118]:
df_one_person = df_one_person.groupby('행정동')['1인가구수'].sum().reset_index()
df_one_person = df_one_person.sort_values('1인가구수', ascending=False)
df_one_person

,행정동,1인가구수
11,석사동,20440
18,퇴계동,19737
20,효자2동,16514
24,후평3동,11280
22,후평1동,11054
15,신사우동,10352
0,강남동,9748
5,동내면,9505
6,동면,7896
23,후평2동,7717


## 춘천시 초등학교와 아동급식카드

### 행정동별 초등학교 위치 수

In [137]:
csv_path_elm = '/content/drive/MyDrive/town_cc_big_data/data/강원도교육청_춘천시초등학교_통학구역_20220731.csv'

In [138]:
df_elm = pd.read_csv(csv_path_elm, encoding='euc-kr')

In [139]:
df_elm = df_elm.groupby('행정동명')['관할학교명'].count().reset_index()
df_elm.columns = ['행정동명', '초등학교 수']

In [140]:
df_elm = df_elm.sort_values('초등학교 수', ascending=False)
df_elm

,행정동명,초등학교 수
18,퇴계동,16
11,석사동,7
14,신북읍,4
12,소양동,4
26,후평2동,4
4,남산면,4
23,효자2동,4
10,서면,4
25,후평1동,3
22,효자1동,3


### 아동급식카드 가맹점 수

In [177]:
csv_path_store = '/content/drive/MyDrive/town_cc_big_data/data/강원도 춘천시_아동급식카드 가맹점_20220608.csv'

In [178]:
df_store = pd.read_csv(csv_path_store, encoding='euc-kr')

In [179]:
df_store = df_store[['소재지지번주소', '가맹점명']]

In [180]:
for index, row in df_store.iterrows():
    address = row['소재지지번주소']
    split_data = address.split()
    if len(split_data) >= 3:
        new_data = split_data[2]
        df_store.at[index, '소재지지번주소'] = new_data

df_store = df_store.groupby('소재지지번주소').size().reset_index(name='가맹점수')
df_store = df_store.sort_values('가맹점수', ascending=False)
df_store = df_store.rename(columns={'가맹점수': '가맹점 수'})
df_store

,소재지지번주소,가맹점 수
38,퇴계동,580
40,후평동,545
15,석사동,534
39,효자동,525
6,동내면,273
7,동면,253
28,온의동,215
30,우두동,156
5,남산면,148
24,신북읍,111


초등학교가 4개 정도 있는 구역에서 가맹점이 별로 없는 구역이 있음.    
그 구역을 중점적으로 봐야 할 듯